## Рекомендательные системы

##### Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

In [1]:
%pylab inline
import pandas as pd
import copy
from collections import Counter

Populating the interactive namespace from numpy and matplotlib


In [2]:
sessions = pd.read_csv('./coursera_sessions_train.txt',delimiter=';', header=None,names=['viewed','bought'])

In [3]:
sessions.info()
sessions.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
viewed    50000 non-null object
bought    3608 non-null object
dtypes: object(2)
memory usage: 781.3+ KB


,viewed,bought
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN
5,42,NaN
6,"47,48,49",NaN
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63"
8,"71,72,73,74",NaN
9,"76,77,78",NaN


##### Важно:

    Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
    Если товар не встречался в обучающей выборке, его популярность равна 0.
    Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
    Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

##### Задание

    На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
    Реализуйте два алгоритма рекомендаций:

    сортировка просмотренных id по популярности (частота появления в просмотренных),
    сортировка просмотренных id по покупаемости (частота появления в покупках).

### Создаем словари частот, парсим в строки элементы сессии, уникализируем эти элементы

In [6]:
viewed = copy.copy(sessions.viewed.values)
bought = copy.copy(sessions.bought.fillna(-1).values)
viewed_dic = []
bought_dic = []
for idx, item  in enumerate(viewed):
    viewed[idx] = viewed[idx].split(",")
    viewed_dic  += viewed[idx]
    viewed[idx] = pd.unique(viewed[idx])
viewed_dic = Counter(viewed_dic)

for idx, item  in enumerate(bought):
        if(bought[idx] != -1):
            bought[idx] = bought[idx].split(",")
            bought_dic  += bought[idx]
            bought[idx] = pd.unique(bought[idx])
bought_dic = Counter(bought_dic)


In [8]:
#viewed[7] = sorted(list(viewed[7]), key=lambda x: viewed_dic[x], reverse=True)
#bought[7] = sorted(list(bought[7]), key=lambda x: bought_dic[x], reverse=True)

### Получаем рекомендации по переданным словарям и сессиям

In [19]:
def recommend_t(viewed_dic, viewed, bought_dic = 0, bought = 0,i=0):
    print(viewed_dic[str(i)],viewed[i],bought_dic[str(i)],bought[i])
    
    k1_v,k1_b,k5_v,k5_b = [],[],[],[]
    sorted_viewed, sorted_bought = [],[]
    
    for idx,item in enumerate(viewed):
        sorted_viewed.append(sorted(list(viewed[idx]), key=lambda x: viewed_dic[x], reverse=True))
        k1_v.append(sorted_viewed[idx][0])
        k5_v.append(sorted_viewed[idx][:5])
    
    if(bought_dic != 0):
        for idx,item in enumerate(bought):
            try:
                sorted_bought.append(sorted(list(bought[idx]), key=lambda x: bought_dic[x], reverse=True))
                k1_b.append(sorted_bought[-1][0])
                k5_b.append(sorted_bought[-1][:5])
                
                #add to top5 from views
                if((len(k5_b[-1]) < 5) & (len(k5_v) != 0)):
                    continue
                
            except:
                sorted_bought.append([-1])
                k1_b.append([-1])
                k5_b.append([-1])
                
    
    
    return {'k1_v':k1_v,'k5_v':k5_v,'sorted_viewed':sorted_viewed,
            'k1_b':k1_b,'k5_b':k5_b,'sorted_bought':sorted_bought}
    
a = recommend_t(viewed_dic, viewed, bought_dic,bought,7)

312 ['59' '60' '61' '62' '63' '64' '65' '66' '67' '68'] 2 ['67' '60' '63']


In [20]:
a['k5_b']

[[-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 ['67', '60', '63'],
 [-1],
 [-1],
 ['86'],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 ['199'],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 ['303'],
 [-1],
 [-1],
 ['352'],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 ['519'],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 ['605', '602', '606', '603', '604'],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 ['690', '688'],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 ['851'],
 [-1],
 [-1],
 [-1],
 ['879'],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 ['1118'],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1